In [1]:
import pandas as pd
import io

In [13]:
#Read CSV file to Pandas dataframe and normalize all the columns

df = pd.read_csv('https://raw.githubusercontent.com/NhatMinh2910/Business-Sales-Analyzing/refs/heads/main/Business_sales_EDA.csv')

df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)

df.head()

,product_id,product_position,promotion,product_category,seasonal,sales_volume,brand,url,name,description,price,currency,terms,section,season,material,origin
0,185102,Aisle,Yes,clothing,Yes,1243,Zara,https://www.zara.com/us/en/basic-puffer-jacket...,BASIC PUFFER JACKET,Puffer jacket made of tear-resistant ripstop f...,78.99,USD,jackets,MAN,Winter,Polyester,Brazil
1,188771,Aisle,Yes,clothing,No,1429,Zara,https://www.zara.com/us/en/tuxedo-jacket-p0889...,TUXEDO JACKET,Straight fit blazer. Pointed lapel collar and ...,14.99,USD,jackets,MAN,Autumn,Cotton,Turkey
2,180176,End-cap,Yes,clothing,Yes,1168,Zara,https://www.zara.com/us/en/slim-fit-suit-jacke...,SLIM FIT SUIT JACKET,Slim fit jacket. Notched lapel collar. Long sl...,71.95,USD,jackets,WOMAN,Autumn,Polyester,Morocco
3,112917,Aisle,Yes,clothing,No,1348,Zara,https://www.zara.com/us/en/stretch-suit-jacket...,STRETCH SUIT JACKET,Slim fit jacket made of viscose blend fabric. ...,30.99,USD,jackets,MAN,Spring,Polyester,China
4,192936,End-cap,Yes,clothing,Yes,1602,Zara,https://www.zara.com/us/en/double-faced-jacket...,DOUBLE FACED JACKET,Jacket made of faux leather faux shearling wit...,22.99,USD,jackets,WOMAN,Winter,Wool Blend,China


# **Categorize columns**

---



*   **'product_id'** - Identifier
* **'product_position'** - Position/ Exposure
* **'promotion', 'seasonal'** - Promotion/ Marketing
* **'product_category', 'name', 'description', 'terms', 'brand', 'material', 'origin', 'section'** - Product Information
* **'sales_volume', 'price', 'currency'** - Sales Metrics
*  **'season', 'url'** - Metadata



In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20252 entries, 0 to 20251
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product_id        20252 non-null  int64  
 1   product_position  20252 non-null  object 
 2   promotion         20252 non-null  object 
 3   product_category  20252 non-null  object 
 4   seasonal          20252 non-null  object 
 5   sales_volume      20252 non-null  int64  
 6   brand             20252 non-null  object 
 7   url               20252 non-null  object 
 8   name              20251 non-null  object 
 9   description       20250 non-null  object 
 10  price             20252 non-null  float64
 11  currency          20252 non-null  object 
 12  terms             20252 non-null  object 
 13  section           20252 non-null  object 
 14  season            20252 non-null  object 
 15  material          20252 non-null  object 
 16  origin            20252 non-null  object

In [15]:
df["sales_volume"] = pd.to_numeric(df["sales_volume"], errors="coerce")
df["price"] = pd.to_numeric(df["price"], errors="coerce")

In [16]:
df.isnull().mean().sort_values(ascending=False)

,0
description,0.000099
name,0.000049
product_id,0.000000
product_category,0.000000
product_position,0.000000
seasonal,0.000000
sales_volume,0.000000
brand,0.000000
promotion,0.000000
url,0.000000


# **Strategies in handling missing values**

---


*   product_id: Do not accept missing value -> Drop row
* sales_volume: Fill with 0
* price: Fill with category median
* brand: Fill "Unknown"
* material/ origin: Fill with "Not specified"




In [17]:
df = df.dropna(subset=["product_id"])
df["brand"] = df["brand"].fillna("Unknown")
df["material"] = df["material"].fillna("Not specified")

In [18]:
#Check for productID duplicates (Drop if there are any duplicates)
df["product_id"].duplicated().sum()

np.int64(0)

In [19]:
#Validate numeric features
#Sales & Price must be > 0
df = df[df["sales_volume"] > 0]
df = df[df["price"] > 0]
#Look for any outlier
df["price"].describe()
df["sales_volume"].describe()

,sales_volume
count,20252.000000
mean,1097.400454
std,298.234609
min,518.000000
25%,849.000000
50%,990.000000
75%,1364.250000
max,1940.000000


In [20]:
#Normalize Promotion and Seasonal values to boolean values
df["promotion"].value_counts()
df["seasonal"].value_counts()
df["promotion"] = df["promotion"].str.lower().isin(["yes", "true", "1"])
df["seasonal"] = df["seasonal"].str.lower().isin(["yes", "true", "1"])
df.head(10)

,product_id,product_position,promotion,product_category,seasonal,sales_volume,brand,url,name,description,price,currency,terms,section,season,material,origin
0,185102,Aisle,True,clothing,True,1243,Zara,https://www.zara.com/us/en/basic-puffer-jacket...,BASIC PUFFER JACKET,Puffer jacket made of tear-resistant ripstop f...,78.99,USD,jackets,MAN,Winter,Polyester,Brazil
1,188771,Aisle,True,clothing,False,1429,Zara,https://www.zara.com/us/en/tuxedo-jacket-p0889...,TUXEDO JACKET,Straight fit blazer. Pointed lapel collar and ...,14.99,USD,jackets,MAN,Autumn,Cotton,Turkey
2,180176,End-cap,True,clothing,True,1168,Zara,https://www.zara.com/us/en/slim-fit-suit-jacke...,SLIM FIT SUIT JACKET,Slim fit jacket. Notched lapel collar. Long sl...,71.95,USD,jackets,WOMAN,Autumn,Polyester,Morocco
3,112917,Aisle,True,clothing,False,1348,Zara,https://www.zara.com/us/en/stretch-suit-jacket...,STRETCH SUIT JACKET,Slim fit jacket made of viscose blend fabric. ...,30.99,USD,jackets,MAN,Spring,Polyester,China
4,192936,End-cap,True,clothing,True,1602,Zara,https://www.zara.com/us/en/double-faced-jacket...,DOUBLE FACED JACKET,Jacket made of faux leather faux shearling wit...,22.99,USD,jackets,WOMAN,Winter,Wool Blend,China
5,117590,End-cap,True,clothing,True,1282,Zara,https://www.zara.com/us/en/contrasting-collar-...,CONTRASTING COLLAR JACKET,Relaxed fit jacket. Contrasting lapel collar a...,25.95,USD,jackets,WOMAN,Autumn,Cotton,Turkey
6,189118,Front of Store,False,clothing,False,688,Zara,https://www.zara.com/us/en/faux-leather-puffer...,FAUX LEATHER PUFFER JACKET,Faux leather puffer jacket. High collar and lo...,87.99,USD,jackets,MAN,Winter,Acrylic,China
7,182157,Aisle,True,clothing,True,1711,Zara,https://www.zara.com/us/en/suit-jacket-in-100-...,SUIT JACKET IN 100% LINEN,Straight fit blazer made of linen. Notched lap...,24.00,USD,jackets,WOMAN,Summer,Cotton,Portugal
8,141861,Aisle,False,clothing,True,857,Zara,https://www.zara.com/us/en/100-wool-suit-jacke...,100% WOOL SUIT JACKET,Jacket made of Italian wool. Pointed lapel col...,32.95,USD,jackets,WOMAN,Winter,Wool,India
9,137121,Aisle,False,clothing,False,769,Zara,https://www.zara.com/us/en/100-feather-fill-pu...,100% FEATHER FILL PUFFER JACKET,Puffer jacket made of shiny finish technical f...,39.95,USD,jackets,MAN,Autumn,Wool Blend,China


In [21]:
#Categorical cleaning (Product Category, Brand, Section, Season) - Change all string to lowercase
cat_cols = ["product_category", "brand", "section", "season"]

for col in cat_cols:
    df[col] = df[col].str.lower().str.strip()

In [22]:
#Check if there are any different currencies used
df["currency"].value_counts()

,count
currency,
USD,20252


In [24]:
#Export the final processed dataset to a new csv file
df.to_csv("Business-sales-EDA-processed.csv", index=False)